In [1]:
# pip install pandas xlrd
import pandas as pd # We import pandas to work with dataframes.
import numpy as np # We import numpy to work with matrix.
import xlrd # We import xlrd because pandas need it due to our original data-table is in a old version
import re # We import regex library for searching patterns. 

# We load the data-table excel from the url.
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
data_df = pd.read_excel(url)

In [2]:
# We attach to our main a python document which is keeping the main function we need to use. 
import sys
sys.path.append("SHARKATTACKSPROYECT/")
from functions import *


In [ ]:
# We are taking a snapshot about our data panel in order to see to what we are facing.

#data_df.shape

In [ ]:
# It seems before step is enough to take a complete snapshot of our data. Lets go with another method.

#pd.options.display.max_columns # We get access to configure display options from Pandas and configure max columns to show.

In [ ]:
# Once we've seen the columns distribution, we gonna face the date types of each column to know how start to work with each one.
# We need to know how many Na/Non Values we have in data.
#data_df.info()

# YEAR AND DATE COLUMNS

In [ ]:
# In column YEAR, lets see how many unique values it contains: 
# data_df.Year.unique()

In [3]:
# There are some years with no values, so we will try if we can confirm the year though column Date. 
data_df[['Date','Year']][data_df.Year == 0]

,Date,Year
6815,Ca. 214 B.C.,0.0
6816,Ca. 336.B.C..,0.0
6817,Ca. 493 B.C.,0.0
6818,Ca. 725 B.C.,0.0
6819,Ca. 1010 BC,0.0
...,...,...
6939,Before 1903,0.0
6940,Before 1903,0.0
6941,1900-1905,0.0
6942,1883-1889,0.0


In [4]:
# Lets try to aply some functions in order to rescue these years:
'''Buscamos mediante un regex, para que nos busque los años numéricos'''

data_df.Date = data_df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(data_df.Date.unique())

['15 Mar 2024',
 '04 Mar 2024',
 '02 Mar-2024',
 '25 Feb-2024',
 '14 Feb-2024',
 '04-Feb-2024',
 '29 Jan-2024',
 '15 Jan-2024',
 '09-Jan-2024',
 '05-Jan-2024',
 '30 Dec-2023',
 '29 Dec-2023',
 '28 Dec-2023',
 '25 Dec-2023',
 '24-Dec-2023',
 '18 Dec-2023',
 '14  Dec-2023',
 '08 Dec-2023',
 '04 Dec-2023',
 '02 Dec-2023',
 '30 Nov-2023',
 '21 Nov-2023',
 '10 Nov-2023',
 '05 Nov-2023',
 '03 Nov-2023',
 '02 Nov-2023',
 '31 Oct-2023',
 '25 Oct 2023',
 '23 Oct-2023',
 '21 Oct-2023',
 '20 Oct 2023',
 '15 Oct 2023',
 '13 Oct-2023',
 '02 Oct-2023',
 '01 Oct-2023',
 '18 Sep-2023',
 '13 Sep-2023',
 '12 Sep-2023',
 '11 Sep-2023',
 '09 Sep- 2023',
 '06-Sep-2023',
 '04 Sep 2023',
 '25 Aug 2023',
 '23 Aug-2023',
 '22 Aug-2023',
 '21 Aug-2023',
 '16 Aug-2023',
 '12 Aug-2023',
 '07 Aug-2023',
 '31 Jul-2023',
 '30 Jul-2023',
 '29 Jul-2023',
 '24 Jul-2023',
 '22 Jul-2023',
 '21 Jul-2023',
 '18 Jul-2023',
 '15 Jul-2023',
 '14 Jul-2023',
 '04 Jul-2023',
 '03 Jul-2023',
 '23-Jun-2023',
 '14-June 2023',
 '08-

In [5]:
# Aunque no salen en los uniques hay muchos Dates que salen como rangos, o datos de antes de Cristo.
# Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:
salvables = data_df.loc[(data_df["Year"] == 0) & (data_df["Date"] != np.nan)]
salvables.shape

(129, 23)

In [6]:
# Llamamos a las funciones definidas en el cleanin_functions.py, mas concretamente a rescatar 
# fechas que aplica 3 funciones secuencialmente donde coge por orden los que contienen BC, 
# los que son fechas sueltas (tipo Before YYYY) y luego los intervalos de los cuales saca la media, 
# para rellenar los datos de la columna Year para esos valores de Dates 
from functions import *


In [ ]:
# Eliminamos todas las filas con valores NA

#data_df_year.dropna(axis=0, inplace= True, how='all')

In [7]:
# Borrar años inferiores al 1700 o superiores al 2024

indexYear = data_df[ (data_df['Year'] <= 1700)].index

data_df.drop(indexYear , inplace=True)

In [8]:

indexYear2 = data_df[ (data_df['Year'] >= 2024)].index

data_df.drop(indexYear2 , inplace=True)

In [ ]:
data_df.Year.unique()

In [10]:
data_df.Year = data_df.Date.apply(rescatar_fechas)

In [12]:
data_df.Year.dropna(axis=0, inplace= True, how='all')

In [13]:
data_df.Year.unique()

array(['2023', '2022', '2021', '2020', nan, '2010', '2019', '2018',
       '1018', '2017', '2016', '2015', '2014', '2013', '2012', '2011',
       '2008', '2009', '2006', '2007', '2005', '2004', '2003', '2002',
       '2001', '2000', '1999', '1998', '1997', '1996', '1995', '1984',
       '1994', '1993', '1992', '1991', '1990', '1989', '1988', '1987',
       '1986', '1985', '1983', '1982', '1962', '1981', '1980', '1979',
       '1978', '1977', '1976', '1975', '1974', '1973', '1972', '1971',
       '1970', '1969', '1968', '1967', '1966', '1965', '1964', '1954',
       '1963', '1961', '1960', '1959', '2176', '1958', '1957', '1956',
       '1955', '1953', '1952', '1951', '1950', '1949', '1948', '1848',
       '1947', '1946', '1945', '1944', '1943', '1942', '1941', '1940',
       '1939', '1938', '1937', '1936', '1935', '1934', '1933', '1932',
       '1931', '1930', '1929', '1928', '1927', '1926', '1925', '1924',
       '1923', '1922', '1921', '1920', '1919', '1918', '1917', '1916',
       '1

In [ ]:
# Lets see to get a general idea about the problem with Na in order to clean approaching. We get percentages.

Na_amount = round(data_df.isna().sum()*100/len(data_df), 2)
Na_amount.sort_values(ascending = False)

In [11]:
# We see there are some columns with a too high percentage of Nan Values.
# A logical idea we can conclude, it is those columns with a high percentage of nan values are useless due to missinformation given.
# So We decide to drop columns named "Unnamed 21" and "Unnamed 22".





In [ ]:


df.dropna(axis=0, inplace= True, thresh=int(24*0.8))

In [ ]:
# Análisis de los datos de fecha

#Columna Date

#Vemos los valores únicos
data_df.Year.unique()

In [ ]:
# Tenemos valores muy raros, como año 0, 5, 77. Por cuestiones históricas, decidimos dropear todo lo anterior al año 1800.

# Como tenemos otra columna (date) que también nos da años, intentamos si podemos rescatar datos: 

data_df[['Date','Year']][data_df.Year == 0] #81 fechas que podemos rescatar a través de date